In [182]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.naive_bayes import MultinomialNB

# 2.1 Построение модели классификации

In [183]:
df = pd.read_pickle('habr.pickle')
df.head()

,name,rating,date,Vector,nominations,text_lemtize
0,0,None,0,"[-0.14686837386151166, 0.23498939817841866, 0....",0,"[декабрь, состояться, церемония, награждение, ..."
1,1,None,0,"[-0.14686837386151166, 0.23498939817841866, 0....",0,"[декабрь, состояться, церемония, награждение, ..."
2,1,None,1,"[-0.3698977214370539, -0.3698977214370539, -0....",0,"[из, совместный, исследование, издатель, компь..."
3,1,None,1,"[-0.33776026473659926, 0.08444006618414981, -0...",0,"[по, статистика, половина, год, впервые, топ, ..."
4,1,None,2,"[-0.48428058736275703, -0.12107014684068926, -...",0,"[компания, провести, совместный, исследование,..."


In [184]:
txt_train_list = []
txt_train = ''
for i in df['text_lemtize']:
    txt_train = ''
    for j in i:
        txt_train = txt_train + ' ' + j
    txt_train_list.append(txt_train)
df['text_lemtize'] = txt_train_list

In [185]:
df = df.drop(['rating'], axis=1) 
df = df.drop('Vector', axis = 1)
df.head()

,name,date,nominations,text_lemtize
0,0,0,0,декабрь состояться церемония награждение лаур...
1,1,0,0,декабрь состояться церемония награждение лаур...
2,1,1,0,из совместный исследование издатель компьютер...
3,1,1,0,по статистика половина год впервые топ самый ...
4,1,2,0,компания провести совместный исследование евр...


### Разделим выборку на обучающую и тестовую

In [186]:
KNN_model = KNeighborsClassifier(n_neighbors=5)
df['name'] = pd.factorize(df['name'])[0]
X = df['text_lemtize']
y = df['nominations']

На тестовые значения отведём только 1/5 от всего набора. 

В переменную X введём все данные кроме целевой переменной, в y - только целевую переменную.

In [187]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=27)

### Обучим нашу модель на классификации Naive Bayes

Наивный байесовский классификатор (Naive Bayes) — это алгоритм машинного обучения, предназначенный для многоклассовой классификации данных с независимыми признаками. За один проход вычисляется условная вероятность каждого признака

In [191]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [192]:
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [193]:
y_pred = nb.predict(X_test)

### Реализуем метрику. Посмотрим точность модели на классификации Naive Bayes

Accuracy - это метрика, показывающая отношение числа правильно угаданных классов к общему количеству примеров. Данная метрика вполне реализует и показывает нам точность нашей модели, учитывая описанный способ её оценки.

In [194]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [196]:
print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.5739644970414202


### Обучим модель на линейной классификации SGDClassifier

Этот оценщик реализует регуляризованные линейные модели со стохастическим градиентным спуском (SGD): градиент потерь оценивается для каждой выборки за раз, и модель обновляется по ходу работы с графиком убывающей прочности (он же скорость обучения).

In [219]:
from sklearn.linear_model import SGDClassifier

In [220]:
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier()),
               ])

In [221]:
sgd.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier())])

### Посмотрим точность модели на классификации SGDClassifier

In [222]:
y_pred = sgd.predict(X_test)

In [223]:
print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.7248520710059172


### Обучим модель на классификации логистической регресии LogisticRegression

Логистическая регрессия - это процесс моделирования вероятности дискретного результата с учетом входной переменной

In [214]:
from sklearn.linear_model import LogisticRegression

In [215]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1000)),
               ])

In [216]:
logreg.fit(X_train, y_train)

C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1000, n_jobs=1))])

### Посмотрим точность модели на классификации LogisticRegression

In [217]:
y_pred = logreg.predict(X_test)

In [218]:
print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.7189349112426036


## Выберем наилучшую модель

Обучив и посмотрел на все модели (их точность по выбранной метрике). Мы сделаем выбор в пользу модели, которая показала наилучшие результаты в выбранной метрике. Наилучшей моделью таким образом оказалась модель, основанная на классификации при помощи SGDClassifier - метрика показала результат в 0.725, что является наивысшим результатом среди всех обученных моделей. Выберем SGDClassifier

# 2.2 Оптимизация модели

### Настроим нашу модель

In [224]:
from sklearn.decomposition import PCA

ValueError: could not convert string to float: ' декабрь состояться церемония награждение лауреат премия рунет год в год премия подать работа вклад развитие российский сегмент сеть интернет девять основный четыре специальный номинация финалист конкурс каждый номинация стать организация лауреат премия рунет стать компания персона причём ранний организатор российский ассоциация электронный коммуникация раэк награждать несколько хороший представитель каждый номинация мнение экспертный сообщество в год обратиться помощь искусственный интеллект добавить один лауреат который выбираться именно мнение ия номинация государство общество победитель благотворительный фонд подарить жизнь ано диалог правительство москва проект цифровой экосистема москва фнс россия программа пушкинский карта ао почта банк номинация сми массовый коммуникация телеканал медиахолдинг ао коммерсантъ номинация образование кадр правительство москва проект московский электронный школа мэш проект программа сбер ребёнок родите